# Domain Definitions

In [1]:
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, List, Set, Dict, Tuple

In [2]:
@dataclass(frozen=True)
class Item:
    name: str
    value: float
    weight: int
    time_to_steal: int

    @property
    def tts(self):
        return self.time_to_steal

In [3]:
@dataclass(frozen=False)
class Node:
    name: str
    items: Set[Item]

    def __init__(self, name: str):
        self.name = name
        self.items = set()

    def add_item(self, name: str, value: float, weight, tts: int) -> 'Node':
        item = Item(name, value, weight, time_to_steal=tts)
        return self._add_item(item)

    def _add_item(self, item: Item) -> 'Node':
        self.items.add(item)
        return self

    def to_dict(self):
        return {'name': self.name, 'items': self.items}

    def to_dicts(self):
        return [{'name': self.name,
                 'item': item.name,
                 'value': item.value,
                 'weight': item.weight,
                 'tts': item.tts}
                for item in self.items]

    def __hash__(self) -> int:
        return self.name.__hash__()

In [4]:
@dataclass(frozen=True)
class Edge:
    source: Node
    destination: Node
    travelling_time: int
    travelling_cost: float

    @property
    def time(self):
        return self.travelling_time

    @property
    def cost(self):
        return self.travelling_cost

    def to_dict(self):
        return {
            'source': self.source.name,
            'destination': self.destination.name,
            'travelling_time': self.time,
            'travelling_cost': self.cost
        }

In [5]:
class Graph:
    nodes: Dict[str, Node]
    edges: Dict[Tuple[Node, Node], Edge]
    connections: Dict[Node, Set[Edge]]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()
        self.connections = dict()

    def get_node(self, name: str) -> Optional[Node]:
        return self.nodes.get(name)

    def get_all_nodes(self) -> List[Node]:
        return list(self.nodes.values())

    def get_all_edges(self) -> List[Edge]:
        return list(self.edges.values())

    def get_connections_from_node(self, node: Node) -> List[Edge]:
        return self.connections.get(node, [])

    def get_connections_from(self, node_name: str) -> List[Edge]:
        node: Optional[Node] = self.get_node(node_name)
        return self.get_connections_from_node(node)

    def add_edge(self, src: str, dst: str, tt: int, tc: float) -> 'Graph':
        source = self.nodes.get(src, Node(src))
        destination = self.nodes.get(dst, Node(dst))
        edge = Edge(source, destination, travelling_time=tt, travelling_cost=tc)
        self._add_node(source)._add_node(destination)
        self.edges[(source, destination)] = edge
        self.connections[source] = self.connections.get(source, []) + [edge]
        return self

    def add_node(self, node_name: str) -> 'Graph':
        self.nodes[node_name] = Node(node_name)
        return self

    def _add_node(self, node: Node) -> 'Graph':
        self.nodes[node.name] = node
        return self

    def to_pandas_df(self):
        vertexes = [node_item for node in self.get_all_nodes() for node_item in node.to_dicts()]
        edges = [edge.to_dict() for edge in self.get_all_edges()]
        return (pd.DataFrame.from_records(vertexes), pd.DataFrame.from_records(edges))

# Graph Construction

In [6]:
datasets_dir = Path().absolute().parent.joinpath('datasets')
travelling_map = f"{datasets_dir}/travelling_map.csv"
treasure_map = f"{datasets_dir}/treasure_map.csv"

In [7]:
travels = pd.read_csv(travelling_map)
treasures = pd.read_csv(treasure_map)

In [8]:
g = Graph()

In [9]:
travels.apply(lambda row: g.add_edge(src=row['source'],
                                     dst=row['destination'],
                                     tt=row['travelling_time'],
                                     tc=row['travelling_cost'])
                           .add_edge(src=row['destination'],
                                     dst=row['source'],
                                     tt=row['travelling_time'],
                                     tc=row['travelling_cost']), axis=1)

treasures.apply(lambda row: g.get_node(row['city'])
                             .add_item(row['name'],
                                       row['value'],
                                       row['weight'],
                                       row['tts']), axis=1);

# Exploratory Analysis

In [10]:
nodes, edges = g.to_pandas_df()

In [11]:
nodes.head(3)

,name,item,value,weight,tts
0,Santa Paula,Coroa do Rei João II,10000,5,10
1,Campos,Espada sagrada,6500,6,5
2,Riacho de Fevereiro,Cálice do Santo Graal,7000,2,6


In [12]:
edges.head(3)

,source,destination,travelling_time,travelling_cost
0,Escondidos,Santa Paula,6,780
1,Santa Paula,Escondidos,6,780
2,Escondidos,Campos,5,350


# Problem Solving 

In [13]:
class GeneticAlgorithm:
    graph: Graph

    def __init__(self, graph: Graph):
        self.graph = graph

    def mutation(self):
        pass

    def fitness(self):
        pass

    def cross_over(self):
        pass